<a href="https://colab.research.google.com/github/yura-eng/yura2024/blob/main/WorldCloud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
a = 'hello world'
print (a)

hello world


#Worldcloud
## practice
### first:

In [ ]:
# Step 1: Install required libraries in Colab
!pip install gradio wordcloud matplotlib

# Step 2: Import the necessary libraries
import gradio as gr
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import re
import numpy as np
from io import BytesIO
from PIL import Image

# Step 3: Define a function to create the word cloud
def generate_wordcloud(text):
    # Basic text cleaning: remove special characters and limit text size
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove special characters
    if len(text) > 5000:  # Limit the length of the text to prevent errors
        text = text[:5000]

    try:
        # Create a WordCloud object
        wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)

        # Convert the WordCloud to an image format that Gradio can display directly
        image = wordcloud.to_image()
        return image
    except Exception as e:
        print(f"Error generating word cloud: {e}")
        return None

# Step 4: Set up the Gradio interface
interface = gr.Interface(fn=generate_wordcloud,
                         inputs="text",
                         outputs="image",
                         title="Word Cloud Generator",
                         description="Enter some text to generate a word cloud of the most common words.")

# Step 5: Launch the app
interface.launch()

#Sentence-by-sentence Read -Aloud practice


In [ ]:
!pip install gradio gtts nltk pypng

In [ ]:
import gradio as gr
from gtts import gTTS
from nltk import tokenize
import os

# Import necessary nltk libraries
import nltk
nltk.download('punkt')

# Global variable to store sentences and the entire text
sentences = []
full_text = ""

# Function to process text and generate sentence options
def process_text(mytext):
    global sentences, full_text
    full_text = mytext
    sentences = tokenize.sent_tokenize(mytext)
    choices = ["Play the whole text"] + [f"{i + 1}. {s}" for i, s in enumerate(sentences)]
    return choices

# Function to generate audio for the selected item
def generate_audio(selected_item):
    global full_text

    if not selected_item:
        return None

    if selected_item == "Play the whole text":
        tts = gTTS(text=full_text, lang='en')
        audio_path = 'full_text.mp3'
        tts.save(audio_path)
        return audio_path

    index = int(selected_item.split('.')[0]) - 1  # Adjust for 0-based index

    if 0 <= index < len(sentences):
        sentence = sentences[index]
        tts = gTTS(text=sentence, lang='en')
        audio_path = f'sentence_{index + 1}.mp3'
        tts.save(audio_path)
        return audio_path
    else:
        return None

# Function to update dropdown choices based on text input
def update_dropdown(mytext):
    choices = process_text(mytext)
    return gr.update(choices=choices)

# Create a Gradio Blocks app
with gr.Blocks() as app:
    with gr.Row():
        textbox = gr.Textbox(label="Enter your text here")
    with gr.Row():
        submit_button = gr.Button("Submit")
    with gr.Row():
        dropdown = gr.Dropdown(choices=[], label="Select Sentence")
    with gr.Row():
        audio_output = gr.Audio(label="Audio of Selected Sentence")

    submit_button.click(fn=update_dropdown, inputs=textbox, outputs=dropdown)
    dropdown.change(fn=generate_audio, inputs=dropdown, outputs=audio_output)

app.launch()